This is my attempt at visualizing the bird sounds. Send these spectrograms through your trusted CNN of choice coupled with a branch for the metadata and you should have a good starting point for a great result in this competition.

This version solves the performance issues I encountered before, so threading has been removed :-)

Ok. First the libraries:

In [ ]:
from pydub import AudioSegment
import matplotlib.pyplot as plt
from scipy.io import wavfile
from tempfile import mkstemp
import numpy as np
from pydub.utils import make_chunks
import glob, os
from PIL import Image

Next the function that actually creates the spectrograms. This is what is called in a separate thread (one at a time in this implementation).

First we read the .ogg file, then we break it up in 5 second chunks in the loop.
I have implemented a check that we don't want to save more than 5 spectrograms from each file to speed it up a bit.
Lastly we use pyplot to render and save the spectrogram.

In [ ]:
def create_spectrograms(filename):
    print('./' + filename)
    mp3_audio = AudioSegment.from_ogg('/' + filename)

    sample = 0

    chunk_length_ms = 5000
    chunks = make_chunks(mp3_audio, chunk_length_ms)

    for i, chunk in enumerate(chunks):
        sample = sample + 1
        if sample > 5:
            sample = 0
            break
        fd, wname = mkstemp('.wav')
        os.unlink(wname)
        chunk.export(wname, format="wav")
        FS, data = wavfile.read(wname)
        os.close(fd)
        
        data_size = np.array(data.shape).shape[0]
        if data_size > 1:
            plt.specgram(data[:, 0], Fs=FS, NFFT=128, noverlap=0)
        else:
            plt.specgram(data, Fs=FS, NFFT=128, noverlap=0)

        plt.axis('off')
        plt.savefig('spectrograms/' + fnames[5] + '/' + fnames[6].replace('.ogg', '_') + str(i * 5 + 5) + '.png',
                    bbox_inches='tight')
        plt.close()

Ok, now the final part which is the main part of the logic. Here we loop through the files one by one. If we encounter a new directory we create it in the spectrograms folder as well for easy training later. Otherwise it is a simpple matter of creating a thread with create_spectrograms, passing the file name we've reached.

In [ ]:
failed = 0
count = 0
old_path = ''

try:
    os.mkdir('spectrograms/')
except OSError:
    print('Could not create spectrograms directory')

for filename in glob.iglob('/kaggle/input/birdclef-2021/train_short_audio/**', recursive=True):
    if os.path.isfile(filename):
        fnames = filename.split('/')

        if old_path != fnames[5]:
            # THIS IS ONLY TO ENSURE WE JUST LOOK AT THE FIRST FIVE SPECIES. 
            # YOU DON'T WANT THAT IN YOUR CODE FROM HERE...
            count += 1
            if count > 5:
                break
            old_path = fnames[5]
            # TO HERE
                
            try:
                os.mkdir('spectrograms/' + fnames[5])
            except OSError:
                failed = failed + 1
        else:   # THIS IS ONLY TO ENSURE WE JUST LOOK AT THE FIRST SAMPLE OF EACH SPECIES. 
                # YOU DON'T WANT THAT IN YOUR CODE. FROM HERE...
            continue
            # TO HERE

        create_spectrograms(filename)

print('All done...')

So now we can see the intricate details of the bird song (and train our network in recognizing these details).

Compare for example the call of Bonaparte's Gull:

In [ ]:
plt.imshow(Image.open('spectrograms/bongul/XC516224_20.png'))

With that of the Ruby-throated Hummingbird:

In [ ]:
plt.imshow(Image.open('spectrograms/rthhum/XC319184_10.png'))